# Check RSAM & DSAR arryas

I used this script when I recognized, that I did not save the vertical component. Instad I saved the east component. I also used it for testing, if a stream is realy empty or if it is simply empty due to a problem.

In [1]:
import numpy as np
import pandas as pd
import obspy
from obspy import UTCDateTime
import obspy.signal.filter
import sys           
sys.path.append("/data/wsd01/pnwstore/")
from pnwstore.mseed import WaveformClient
client = WaveformClient()
import datetime
import time
import scipy
import glob
import os
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

## Functions

In [2]:
def preprocessing(year,jday, net, sta, cha):

    try:
        # this stream will be used for RSAM and DSAR calculations
        #st = obspy.read('/1-fnp/pnwstore1/p-wd05/PNW2004/UW/2004/{}/EDM.UW.2004.{}'.format(jday,jday))
        st = client.get_waveforms(network=net, station=sta, channel=cha,
                                       year='{}'.format(year), doy='{}'.format(jday))

        st.detrend('linear')
        st.taper(max_percentage=None,max_length=5, type='hann') #max_length in sec
        
        # correct insrument response
        inv = obspy.read_inventory('/auto/pnwstore1-wd11/PNWStationXML/{}/{}.{}.xml'.format(net,net,sta))
        pre_filt = [1e-3, 5e-2, 45, 50]
        water_level = 60
        
        for tr in st:
            tr.remove_response(inventory=inv, zero_mean=True,taper=True, taper_fraction=0.05,
                                      pre_filt=pre_filt, output="VEL", water_level=water_level,
                                      plot=False)

            # correct positive dip
            dip = inv.get_orientation(tr.id, datetime=tr.stats.starttime)['dip']
            if dip > 0:
                tr.data *= -1
#         st.merge(fill_value=0)
        print(':) year={}, jday={}, net={}, sta={}, cha={}'.format(year,jday, net, sta, cha))
    except:
        print('pass station {} day {}'.format(sta,jday))
    return(st)
    
def noise_analysis(data, datas, samp_rate, N, Nm):
    rms_list = []
    rmes_list = []
    pgv_list = []
    pga_list = []

    for i in np.arange(0,Nm,N): # start samples (sample, where next 10min starts)
        data_cut = data[i:i+N-1]

        rms = np.sqrt(np.mean(data_cut**2))
        rmes = np.sqrt(np.median(data_cut**2))
        pgv = max(abs(data_cut))

        data_acc = (data_cut.copy()[:-1] - data_cut.copy()[1:]) / (1/samp_rate)
        pga = max(abs(data_acc))

        rms_list.append(rms)
        rmes_list.append(rmes)
        pgv_list.append(pgv)
        pga_list.append(pga)
    datas.append(np.array(rms_list))
    datas.append(np.array(rmes_list))
    datas.append(np.array(pgv_list))
    datas.append(np.array(pga_list))
    return (datas)
    
def RSAM(data, samp_rate, datas, freq, Nm, N):
    filtered_data = obspy.signal.filter.bandpass(data, freq[0], freq[1], samp_rate)
    filtered_data = abs(filtered_data[:Nm])
    datas.append(filtered_data.reshape(-1,N).mean(axis=-1)*1.e9)
    return(datas)

def VSAR(data, samp_rate, datas, freqs_names, freqs, Nm, N):
    # compute ratio between different velocities
    data -= np.mean(data) # detrend('mean')
    j = freqs_names.index('mf')
    mfd = obspy.signal.filter.bandpass(data, freqs[j][0], freqs[j][1], samp_rate)
    mfd = abs(mfd[:Nm])
    mfd = mfd.reshape(-1,N).mean(axis=-1)
    j = freqs_names.index('hf')
    hfd = obspy.signal.filter.bandpass(data, freqs[j][0], freqs[j][1], samp_rate)
    hfd = abs(hfd[:Nm])
    hfd = hfd.reshape(-1,N).mean(axis=-1)
    vsar = mfd/hfd
    datas.append(vsar)
    return(datas)

def DSAR(data, samp_rate, datas, freqs_names, freqs, Nm, N):
    # compute dsar
    data = scipy.integrate.cumtrapz(data, dx=1./100, initial=0) # vel to disp
    data -= np.mean(data) # detrend('mean')
    j = freqs_names.index('mf')
    mfd = obspy.signal.filter.bandpass(data, freqs[j][0], freqs[j][1], samp_rate)
    mfd = abs(mfd[:Nm])
    mfd = mfd.reshape(-1,N).mean(axis=-1)
    j = freqs_names.index('hf')
    hfd = obspy.signal.filter.bandpass(data, freqs[j][0], freqs[j][1], samp_rate)
    hfd = abs(hfd[:Nm])
    hfd = hfd.reshape(-1,N).mean(axis=-1)
    dsar = mfd/hfd
    datas.append(dsar)
    return(datas)

def lDSAR(data, samp_rate, datas, freqs_names, freqs, Nm, N):
    # compute dsar for low frequencies
    data = scipy.integrate.cumtrapz(data, dx=1./100, initial=0) # vel to disp
    data -= np.mean(data) # detrend('mean')
    j = freqs_names.index('rsam')
    lfd = obspy.signal.filter.bandpass(data, freqs[j][0], freqs[j][1], samp_rate)
    lfd = abs(lfd[:Nm])
    lfd = lfd.reshape(-1,N).mean(axis=-1)
    j = freqs_names.index('mf')
    mfd = obspy.signal.filter.bandpass(data, freqs[j][0], freqs[j][1], samp_rate)
    mfd = abs(mfd[:Nm])
    mfd = mfd.reshape(-1,N).mean(axis=-1)
    ldsar = lfd/mfd
    datas.append(ldsar)
    return(datas)

def nDSAR(datas):
    dsar = datas[3]
    ndsar = dsar/sc.stats.zscore(dsar)
    datas.append(ndsar)
    return(datas)
    
# creates a df for each trace and append this df to a daily df
# def create_df(datas, ti, freqs_names, df):
#     datas = np.array(datas)
#     time = [(ti+j*600).datetime for j in range(datas.shape[1])]
#     df_tr = pd.DataFrame(zip(*datas), columns=freqs_names, index=pd.Series(time))
#     df = pd.concat([df, df_tr])
#     return(df)    

def create_df(datas, ti, freqs_names, df):
    datas = np.array(datas)
    time = [(ti+j*600).datetime for j in range(datas.shape[1])]
    df_tr = pd.DataFrame(zip(*datas), columns=freqs_names+['rms','rmes','pgv','pga'], index=pd.Series(time))
    df = pd.concat([df, df_tr])
    return(df)

## Script Start

In [2]:
year = 1981
sta = 'STD'

### Read the npz file

In [6]:
# all days (one year) of one station
sta_files = sorted(glob.glob('/data/wsd03/data_manuela/MtStHelens/RSAM_DSAR/tmp_{}/{}/*.npz'.format(year,sta)))
li = []
for filename in sta_files:
    frame = pd.read_csv(filename) # read one day
    li.append(frame) # all days (one year) of one station

df = pd.concat(li, axis=0, ignore_index=True) # list to df
df.set_index('time',inplace=True) # index to time
df.index = pd.to_datetime(df.index).tz_localize(None) # time to datetime
df.sort_index() # sort by date (index)

df

ValueError: No objects to concatenate

### Read the extended csv file

In [4]:
# read the existing file
ddff = pd.read_csv('../data/{}_{}_extended2.csv'.format(year,sta))
ddff#[862:870] # for a spesific date

FileNotFoundError: [Errno 2] No such file or directory: '../data/1980_CDF_extended2.csv'

In [ ]:
# check only the seismic stream
st = preprocessing(year,7, 'CC', sta, '?HZ')
st

In [ ]:
# check the created data arrays and df
start_time = time.time()
freqs_names = ['rsam','mf','hf','dsar','ldsar', 'vsar']
df = pd.DataFrame(columns=freqs_names)
daysec = 24*3600
freqs = [[2,5], [4.5,8], [8,16]]
        
if len(st)>0: # if stream not empty
#         st.resample(50)
    for tr in st:
        print(tr)
#         tr = st[0]
        datas = []
        data = tr.data
        samp_rate = tr.meta['sampling_rate']
        ti = tr.meta['starttime']
        # round start time to nearest 10 min increment
        tiday = obspy.UTCDateTime("{:d}-{:02d}-{:02d} 00:00:00".format(ti.year, ti.month, ti.day)) # date
        ti = tiday+int(np.round((ti-tiday)/600))*600 # nearest 10 min to starttime
        N = int(600*samp_rate)    # 10 minute windows in seconds
        Nm = int(N*np.floor(len(data)/N)) # np.floor rounds always to the smaller number
        # seconds per day (86400) * sampling rate (100) -> datapoints per day

        for freq, frequ_name in zip(freqs, freqs_names[:3]):
            datas = RSAM(data, samp_rate, datas, freq, Nm, N) # get RSAM for different frequency bands

        datas = DSAR(data, samp_rate, datas, freqs_names, freqs, Nm, N)
        datas = lDSAR(data, samp_rate, datas, freqs_names, freqs, Nm, N)
        datas = VSAR(data, samp_rate, datas, freqs_names, freqs, Nm, N)
#             datas = nDSAR(datas) # --> add ndsar in freqs_names

        datas = noise_analysis(data, datas, samp_rate, N, Nm)
    
        print(datas)

        df = create_df(datas, ti, freqs_names, df)

    if not os.path.exists(file_path):
        os.makedirs(file_path)

#     df.to_csv(file_path + file_name, index=True, index_label='time')
    print(df)
    print('One day tooks {} seconds.'.format(round(time.time()-start_time),3))